## Run Once

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cpu


### Dataset Creation

In [ ]:
dataset1 = pd.read_csv('/content/drive/My Drive/184A/finalProj/data/processed.cleveland.txt', header = None)
dataset1.columns = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','num']
dataset1 = dataset1.replace('?', np.nan)
dataset1 = dataset1.astype({'age': 'float64','sex': 'category','cp': 'category','trestbps': 'float64','chol': 'float64', 'fbs': 'category', 'restecg': 'category', 'thalach': 'float64', 'exang': 'category', 'oldpeak': 'float64', 'slope': 'category', 'ca': 'category', 'thal': 'category', 'num': 'category'})

dataset2 = pd.read_csv('/content/drive/My Drive/184A/finalProj/data/reprocessed.hungarian.txt', sep=" ", header = None)
dataset2.columns = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','num']
dataset2 = dataset2.replace(-9, np.nan)
dataset2['ca'] = dataset2['ca'].replace(9, np.nan)
dataset2 = dataset2.astype({'age': 'float64','sex': 'category','cp': 'category','trestbps': 'float64','chol': 'float64', 'fbs': 'category', 'restecg': 'category', 'thalach': 'float64', 'exang': 'category', 'oldpeak': 'float64', 'slope': 'category', 'ca': 'category', 'thal': 'category', 'num': 'category'})

dataset3 = pd.read_csv('/content/drive/My Drive/184A/finalProj/data/processed.switzerland.txt', header = None)
dataset3.columns = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','num']
dataset3 = dataset3.replace('?', np.nan)
dataset3['chol'] = dataset3['chol'].replace(0,np.nan)
dataset3 = dataset3.astype({'age': 'float64','sex': 'category','cp': 'category','trestbps': 'float64','chol': 'float64', 'fbs': 'category', 'restecg': 'category', 'thalach': 'float64', 'exang': 'category', 'oldpeak': 'float64', 'slope': 'category', 'ca': 'category', 'thal': 'category', 'num': 'category'})

dataset4 = pd.read_csv('/content/drive/My Drive/184A/finalProj/data/processed.va.txt', header = None)
dataset4.columns = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','num']
dataset4 = dataset4.replace('?', np.nan)
dataset4['chol'] = dataset4['chol'].replace('0',np.nan)
dataset4['trestbps'] = dataset4['trestbps'].replace('0',np.nan)
dataset4 = dataset4.astype({'age': 'float64','sex': 'category','cp': 'category','trestbps': 'float64','chol': 'float64', 'fbs': 'category', 'restecg': 'category', 'thalach': 'float64', 'exang': 'category', 'oldpeak': 'float64', 'slope': 'category', 'ca': 'category', 'thal': 'category', 'num': 'category'})

In [ ]:
datasetAll = pd.concat([dataset1, dataset2, dataset3, dataset4], ignore_index=True, sort=False)

#top 6 features - oldpeak, chol, thalach, age, trestbps, ca
#middle - slope
#bottom - restecg, cp, thal, fbs, exang, sex (lowest)

#drop ca, slope, thal
#(558, 11)
datasetV1 = datasetAll.drop(['slope', 'ca', 'thal'], axis = 1)
datasetV1 = datasetV1.dropna()
datasetV1 = datasetV1.reset_index(drop=True)

#drop ca, thal (will lose 1/3 of data since slope is 1/3 nan)
#(392, 12)
datasetV2 = datasetAll.drop(['ca', 'thal'], axis = 1)
datasetV2 = datasetV2.dropna()
datasetV2 = datasetV2.reset_index(drop=True)

#drop thal (will still lose majority of data since ca had 2/3 nan)
#(297, 13)
datasetV3 = datasetAll.drop(['thal'], axis = 1)
datasetV3 = datasetV3.dropna()
datasetV3 = datasetV3.reset_index(drop=True)

## Print Hider

with HiddenPrints():

    print("This will not be printed")

print("This will be printed as before")

In [ ]:
import os, sys

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

## Undersampling
undersampler(Xtr, Ytr) returns a list with a (Xtr,Ytr) tuple for each undersample

In [ ]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

from collections import Counter
from imblearn.under_sampling import NeighbourhoodCleaningRule, TomekLinks, ClusterCentroids, NearMiss, EditedNearestNeighbours, CondensedNearestNeighbour, OneSidedSelection, RandomUnderSampler
from imblearn.over_sampling import SMOTE

def undersampler(Xtr, Ytr):
  retList = []

  XtrSamp = Xtr
  YtrSamp = Ytr

  print('Original dataset shape %s' % Counter(Ytr))
  print()

  rus = RandomUnderSampler(sampling_strategy='auto')
  X_res, y_res = rus.fit_resample(Xtr, Ytr)
  print('RandomUnderSampler shape %s' % Counter(y_res))
  XtrSamp = X_res
  YtrSamp = y_res
  dataSamp = pd.DataFrame(np.column_stack((XtrSamp,YtrSamp)))
  dataSamp.rename({(inputNum): 'num'}, axis=1, inplace=True)
  dataSamp = shuffle(dataSamp)
  Xtr2 = dataSamp.iloc[:,0:-1]
  Ytr2 = dataSamp.iloc[:,-1]
  print(dataSamp.shape)
  print(dataSamp['num'].value_counts())
  retList.append(tuple((Xtr2,Ytr2)))
  print()

  cnn = CondensedNearestNeighbour(sampling_strategy='auto')
  X_res, y_res = cnn.fit_resample(Xtr, Ytr)
  print('CondensedNearestNeighbour shape %s' % Counter(y_res))
  XtrSamp = X_res
  YtrSamp = y_res
  dataSamp = pd.DataFrame(np.column_stack((XtrSamp,YtrSamp)))
  dataSamp.rename({(inputNum): 'num'}, axis=1, inplace=True)
  dataSamp = shuffle(dataSamp)
  Xtr2 = dataSamp.iloc[:,0:-1]
  Ytr2 = dataSamp.iloc[:,-1]
  print(dataSamp.shape)
  print(dataSamp['num'].value_counts())
  retList.append(tuple((Xtr2,Ytr2)))
  print()

  nm = NearMiss(sampling_strategy='auto')
  X_res, y_res = nm.fit_resample(Xtr, Ytr)
  print('NearMiss shape %s' % Counter(y_res))
  XtrSamp = X_res
  YtrSamp = y_res
  dataSamp = pd.DataFrame(np.column_stack((XtrSamp,YtrSamp)))
  dataSamp.rename({(inputNum): 'num'}, axis=1, inplace=True)
  dataSamp = shuffle(dataSamp)
  Xtr2 = dataSamp.iloc[:,0:-1]
  Ytr2 = dataSamp.iloc[:,-1]
  print(dataSamp.shape)
  print(dataSamp['num'].value_counts())
  retList.append(tuple((Xtr2,Ytr2)))
  print()

  oss = OneSidedSelection(sampling_strategy='auto')
  X_res, y_res = oss.fit_resample(Xtr, Ytr)
  print('OneSidedSelection shape %s' % Counter(y_res))
  XtrSamp = X_res
  YtrSamp = y_res
  dataSamp = pd.DataFrame(np.column_stack((XtrSamp,YtrSamp)))
  dataSamp.rename({(inputNum): 'num'}, axis=1, inplace=True)
  dataSamp = shuffle(dataSamp)
  Xtr2 = dataSamp.iloc[:,0:-1]
  Ytr2 = dataSamp.iloc[:,-1]
  print(dataSamp.shape)
  print(dataSamp['num'].value_counts())
  retList.append(tuple((Xtr2,Ytr2)))
  print()

  cc = ClusterCentroids(sampling_strategy='auto')
  X_res, y_res = cc.fit_resample(Xtr, Ytr)
  print('ClusterCentroids shape %s' % Counter(y_res))
  XtrSamp = X_res
  YtrSamp = y_res
  dataSamp = pd.DataFrame(np.column_stack((XtrSamp,YtrSamp)))
  dataSamp.rename({(inputNum): 'num'}, axis=1, inplace=True)
  dataSamp = shuffle(dataSamp)
  Xtr2 = dataSamp.iloc[:,0:-1]
  Ytr2 = dataSamp.iloc[:,-1]
  print(dataSamp.shape)
  print(dataSamp['num'].value_counts())
  retList.append(tuple((Xtr2,Ytr2)))
  print()

  return retList


## Into DataLoader

retDataLoaders(Xtr, Ytr, Xva, Yva, batch_size = 30)

return tuple((trainloader, valiloader))

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

def retDataLoaders(Xtr,Ytr,Xva,Yva, batch_size = 30):
  pdXtr = Xtr
  pdYtr = Ytr
  pdXva = Xva
  pdYva = Yva

  npXtr = np.array(pdXtr, dtype='float')
  npYtr = np.array(pdYtr, dtype='float')
  npXva = np.array(pdXva, dtype='float')
  npYva = np.array(pdYva, dtype='float')

  tensorXtr = torch.Tensor(npXtr) # transform to torch tensor
  tensorYtr = torch.Tensor(npYtr)
  tensorYtr = tensorYtr.type(torch.LongTensor)
  datasetTr = TensorDataset(tensorXtr, tensorYtr) # create your datset
  trainloader = DataLoader(datasetTr, batch_size, True) # create your dataloader

  tensorXva = torch.Tensor(npXva) # transform to torch tensor
  tensorYva = torch.Tensor(npYva)
  tensorYva = tensorYva.type(torch.LongTensor)
  datasetVa = TensorDataset(tensorXva, tensorYva) # create your datset
  valiloader = DataLoader(datasetVa, batch_size, True) # create your dataloader

  data, labels = next(iter(trainloader))
  print(data.shape, labels.shape)

  return tuple((trainloader, valiloader))

## Model

model = NeuralNet#(input_size, hidden_size, num_classes).to(device)

1,2, or 3 in place of #


In [ ]:
class NeuralNet3(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet3, self).__init__()
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(input_size, hidden_size) #input and first hidden layer
        self.fc2 = nn.Linear(hidden_size, hidden_size) #hidden layer
        self.fc3 = nn.Linear(hidden_size, hidden_size) #hidden layer
        self.fc4 = nn.Linear(hidden_size, num_classes) #output

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.fc4(out)
        return out

class NeuralNet2(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet2, self).__init__()
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(input_size, hidden_size) #input and first hidden layer
        self.fc2 = nn.Linear(hidden_size, hidden_size) #hidden layer
        self.fc3 = nn.Linear(hidden_size, num_classes) #output

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

class NeuralNet1(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet1, self).__init__()
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(input_size, hidden_size) #input and first hidden layer
        self.fc2 = nn.Linear(hidden_size, num_classes) #output

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

## Dataset Selection

In [ ]:
#theDataset = datasetV1 #10 inputs w/o ca, slope, thal
#theDataset = datasetV2 #11 inputs w/o ca, thal
theDataset = datasetV3 #12 inputs w/o thal
inputNum = theDataset.shape[1] -1

print(theDataset.shape)
print(theDataset['num'].value_counts())

from sklearn.utils import shuffle
np.random.seed(0)
theDataset = shuffle(theDataset)
X = theDataset.iloc[:,0:-1]
y = theDataset.iloc[:,-1]
Xtr, Xva, Ytr, Yva = train_test_split(X, y, test_size=0.25, random_state=0, stratify=y)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(Xtr)
Xtr = scaler.transform(Xtr)
Xva = scaler.transform(Xva)

(301, 13)
0.0    161
1.0     56
2.0     36
3.0     35
4.0     13
Name: num, dtype: int64


## Hyperparameters

In [ ]:
# Hyper-parameters
input_size = inputNum
num_classes = (theDataset['num'].value_counts()).size

hiddenArray = [4,8,10,12,14,18]
epochArray = [5,10,20,40,70,100,200,400]
learning_rate = 0.001

### Train and Test

In [ ]:
f = open("NNOutput3.txt", "w")
f.write(f"Undersampler, NN Hidden Layers, Hidden Units, Epochs, Train, Val\n")
NeuralNet = [NeuralNet1, NeuralNet2, NeuralNet3]
with HiddenPrints():
  balancedDataList = undersampler(Xtr, Ytr)
for numUnder, (Xtr, Ytr) in enumerate(balancedDataList):
  print("The", numUnder, "undersampler")
  with HiddenPrints():
    trainloader, valiloader = retDataLoaders(Xtr, Ytr, Xva, Yva, batch_size = 30)
  for layers in range(3):
    print("\tFor a", layers + 1, "layer NN")
    for hidden_size in hiddenArray:
      print("\t\tWith", hidden_size, "units per layer")
      model = (NeuralNet[layers])(input_size, hidden_size, num_classes).to(device)
      #Loss and optimizer
      criterion = nn.CrossEntropyLoss()
      optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

      for num_epochs in epochArray:
        print("\t\t\tWith", num_epochs, "epochs")

        # Train the model
        total_step = len(trainloader)
        for epoch in range(num_epochs):
          for i, (data, target) in enumerate(trainloader):
            # Move tensors to the configured device
            data = data.to(device)
            target = target.to(device)

            # Forward pass
            outputs = model(data)
            loss = criterion(outputs, target)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


        # Test the model on training
        # In test phase, we don't need to compute gradients (for memory efficiency)
        with torch.no_grad():
            correct = 0
            total = 0
            for data, labels in trainloader:
                data = data.to(device)
                labels = labels.to(device)
                outputs = model(data)
                _, predicted = torch.max(outputs.data, 1)
                #print("Predicted:", predicted, "\nLabel:", labels)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            trainCorrect = 100 * correct / total
            #print('Accuracy of the network on the training data: {} %'.format(trainCorrect))

        # Test the model
        # In test phase, we don't need to compute gradients (for memory efficiency)
        with torch.no_grad():
            correct = 0
            total = 0
            for data, labels in valiloader:
                data = data.to(device)
                labels = labels.to(device)
                outputs = model(data)
                _, predicted = torch.max(outputs.data, 1)
                #print("Predicted:", predicted, "\nLabel:", labels)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            valiCorrect =  100 * correct / total
            #print('Accuracy of the network on the test data: {} %'.format(valiCorrect))

        print("\t\t\t\tTraining Accuracy:", trainCorrect)
        print("\t\t\t\tValidation Accuracy:", valiCorrect)

        f.write(f"{numUnder},{layers+1},{hidden_size},{num_epochs},{trainCorrect:.2f},{valiCorrect:.2f}\n")
  #input("Press Enter to continue...")
f.close()

The 0 undersampler
	For a 1 layer NN
		With 4 units per layer
			With 5 epochs
				Training Accuracy: 20.0
				Validation Accuracy: 3.9473684210526314
			With 10 epochs
				Training Accuracy: 20.0
				Validation Accuracy: 3.9473684210526314
			With 20 epochs
				Training Accuracy: 20.0
				Validation Accuracy: 3.9473684210526314
			With 40 epochs
				Training Accuracy: 20.0
				Validation Accuracy: 3.9473684210526314
			With 70 epochs
				Training Accuracy: 28.0
				Validation Accuracy: 18.42105263157895
			With 100 epochs
				Training Accuracy: 42.0
				Validation Accuracy: 44.73684210526316
			With 200 epochs
				Training Accuracy: 46.0
				Validation Accuracy: 47.36842105263158
			With 400 epochs
				Training Accuracy: 50.0
				Validation Accuracy: 48.68421052631579
		With 8 units per layer
			With 5 epochs
				Training Accuracy: 24.0
				Validation Accuracy: 14.473684210526315
			With 10 epochs
				Training Accuracy: 24.0
				Validation Accuracy: 15.789473684210526
			With 20 epoch